In [1]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image


In [2]:
import torch

: 

: 

In [2]:
import cv2

def detect_person(image_path):
    # Cargar el modelo preentrenado de MobileNet-SSD
    prototxt = "../modelos_preentrenados/deploy.prototxt"
    model = "../modelos_preentrenados/mobilenet_iter_73000.caffemodel"
    net = cv2.dnn.readNetFromCaffe(prototxt, model)

    # Cargar la imagen y obtener sus dimensiones
    image = cv2.imread(image_path)
    (h, w) = image.shape[:2]

    # Preprocesar la imagen
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)

    # Pasar la imagen preprocesada a través de la red
    net.setInput(blob)
    detections = net.forward()

    person_detected = False

    # Analizar las detecciones
    for i in range(detections.shape[2]):
        # Obtener la confianza (probabilidad) asociada a la detección
        confidence = detections[0, 0, i, 2]

        # Filtrar detecciones con confianza mayor a un umbral (por ejemplo, 0.5)
        if confidence > 0.75:
            # Obtener el índice de la clase de la detección
            idx = int(detections[0, 0, i, 1])

            # Si la clase es "persona" (clase 15 en COCO dataset)
            if idx == 15:
                person_detected = True
                break

    return person_detected

# Ejemplo de uso
image_path = "../images/imagenpreubaK655-1.jpg"
if detect_person(image_path):
    print("Persona detectada en la imagen")
else:
    print("No se detectaron personas en la imagen")

No se detectaron personas en la imagen


In [3]:
cont = 1
while cont <= 11:
    df = pd.read_csv(f'../data/datos{cont}_cleanRGB.csv')
    
    for i in df.ph_nombre:
        imagen = '../images/'+i
        if detect_person(imagen):
            df.loc[df.ph_nombre == i, 'ph_persona'] = 1

    df.to_csv(f'../data/datos{cont}_cleanRGBCV.csv', index=False)
    cont += 1

In [ ]:

def detect_animals(image_path):
    # Cargar el modelo preentrenado de YOLOv5s
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s.pt')

    # Cargar la imagen
    img = Image.open(image_path)

    # Realizar la detección de objetos
    results = model(img)

    # Clases de animales en el dataset COCO
    animal_classes = [14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

    animal_detected = False

    # Analizar las detecciones
    for det in results.pred[0]:
        class_id = int(det[5])
        confidence = det[4]

        # Filtrar detecciones con confianza mayor a un umbral (por ejemplo, 0.5)
        if confidence > 0.75:
            # Si la clase es un animal
            if class_id in animal_classes:
                animal_detected = True
                break

    return animal_detected



In [ ]:
cont = 1
while cont <= 11:
    df = pd.read_csv(f'../data/datos{cont}_cleanRGB.csv')
    
    for i in df.ph_nombre:
        imagen = '../images/'+i
        if detect_person(imagen):
            df.loc[df.ph_nombre == i, 'ph_animal'] = 1

    df.to_csv(f'../data/datos{cont}_cleanRGBCV2.csv', index=False)
    cont += 1